In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from text_preprocessing import create_songs_for, preprocess_texts, parse_raw_songs
from Midi_preprocessing import preprocess_midi, encode_midi
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Itay
[nltk_data]     Gabbay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
midi_dir = "midi_files"
lyrics_file = "lyrics_train_set.csv"
with open(lyrics_file, 'r') as raw_lyrics:
    raw_songs = raw_lyrics.read().splitlines()

    songs = parse_raw_songs(raw_songs)

    midi_files = all_midi_files(midi_dir)

    sequences = list()
    seq_counts = []
    for i, song in enumerate(songs):
        seq_count = 0
        midi_name = song_midi_filename(song)

        matched_midi_files = [midi_file for midi_file in midi_files
                              if midi_name in midi_file.lower()]

        if len(matched_midi_files) != 1:
            print("OH OH", len(matched_midi_files), song)
            continue

        songs[i]['midi_file'] = matched_midi_files[0]

        if songs[i]['lyrics'].find('&,,,,') == -1:
            songs[i]['lyrics'] = songs[i]['lyrics'] + ' EOS'
        else:
            songs[i]['lyrics'] = songs[i]['lyrics'].replace('&,,,,', ' EOS')

        songs[i]['lyrics'] = ' '.join(nltk.word_tokenize(songs[i]['lyrics']))
        songs[i]['lyrics'] = songs[i]['lyrics'].replace('&', 'EOL')

        songs[i]['lyrics'] = remove_brackets(songs[i]['lyrics'])

        splitted_lyrics = [token for token in nltk.word_tokenize(songs[i]['lyrics']) if token not in string.punctuation]
        
  
        n = 4
        splitted_lyrics = (['<s>'] * (n-1)) + splitted_lyrics
    
        for j in range(0, len(splitted_lyrics) - n):
            sequence = splitted_lyrics[j:j+n]
            sequences.append(sequence)
            seq_count += 1
        
        seq_counts.append(seq_count)
            
            
all_songs_words = ' '.join(np.hstack(sequences).flatten()) + ' eos'
tokenizer = init_tokenizer(all_songs_words)
word_index = tokenizer.word_index


sequences = [np.array(tokenizer.texts_to_sequences(seq)).flatten() for seq in sequences]

sequences = pad_sequences(sequences, maxlen=n)

# split into input and output elements
sequences = np.array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=len(word_index))

embeddings_index = {}
f = open('glove.6B.300d.txt', encoding="utf8")
EMBEDDING_DIM = 300
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
unk_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        unk_words.append(word)

print("Found", len(unk_words) + 1, "Unknown words")
print(unk_words)

NameError: name 'all_midi_files' is not defined

In [6]:
songs = create_songs_for()

text_tokenizer, text_sequences, embedding_matrix = preprocess_texts([song['lyrics'] for song in songs], 200)
midi_tokenizer, midi_sequences = preprocess_midi([song['midi_file'] for song in songs], 300, 1)

Num of unknown words: 568


  0%|▏                                                                                 | 1/606 [00:00<02:47,  3.62it/s]c:\users\itay gabbay\appdata\local\programs\python\python36\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)
100%|████████████████████████████████████████████████████████████████████████████████| 606/606 [12:28<00:00,  2.14s/it]


In [ ]:
n=4
sequences = [np.array(tokenizer.texts_to_sequences(seq)).flatten() for seq in text_sequences]
sequences = pad_sequences(sequences, maxlen=n)